In [204]:
import genomeview
import pyBigWig
import subprocess, os, glob
import pandas as pd
import numpy as np
from IPython.display import display, HTML

def create_pwm(cov_path, min_cov = None, fract_mean_cov = 0.10):
    """returns PWM"""
    rev_comp = {'a':'T', 't':'A', 'c':'G', 'g':'C'}
    qual_dict = {}
    for idx, x in enumerate(range(300)): # ASCII -33 (PHRED)
        qual_dict[chr(x)] = idx-33

    chrom_list = []
    pos_list = []
    ref_list = []
    baseq_list = []
    mapq_list = []
    A_list = []
    T_list = []
    G_list = []
    C_list = []
    cbase_list = []
    
    reads_list = []
    for line in open(cov_path):
        ID,pos,ref,num_reads,bases,base_qual,map_qual = line.split()
        reads_list = reads_list + [int(num_reads)]
    mean_cov = np.mean(reads_list)
    if min_cov == None:
        cov_threshold = int(fract_mean_cov*mean_cov)
    else: 
        cov_threshold = int(min_cov)
    print('coverage_threshold = ', cov_threshold)

    for line in open(cov_path):
        ID,pos,ref,num_reads,bases,base_qual,map_qual = line.split()
        chrom_list = chrom_list + [ID]
        pos_list = pos_list + [pos]
        ref_list = ref_list + [ref]
        baseq_list = baseq_list + [int(np.median([qual_dict[x] for x in base_qual]))]
        mapq_list = mapq_list + [int(np.median([qual_dict[x] for x in map_qual]))]

        if int(num_reads) == 0: # at least 1 read coverage
            A_list = A_list + [0]
            T_list = T_list + [0]
            G_list = G_list + [0]
            C_list = C_list + [0]
            cbase_list = cbase_list + [ref]
        else:
            counts = {'A':0, 'C':0, 'G':0, 'T':0}
            for c in bases:
                if c in {'.', ','}:
                    counts[ref] += 1
                elif c in {'A', 'C', 'G', 'T'}:
                    counts[c] += 1
                elif c in {'a', 'c', 'g', 't'}:
                    counts[rev_comp[c]] += 1
            if sum(counts.values()) == 0: # at least 1 base detected
                A_list = A_list + [0]
                T_list = T_list + [0]
                G_list = G_list + [0]
                C_list = C_list + [0]
                cbase_list = cbase_list + [ref]
            else:
                A_list = A_list + [counts['A']]
                T_list = T_list + [counts['T']]
                G_list = G_list + [counts['G']]
                C_list = C_list + [counts['C']]
                cbase = [key for key,value in counts.items() if value >= cov_threshold]
                if len(cbase) > 1:
                    cbase = ['/'.join(cbase)]
                elif len(cbase) == 0:
                    cbase = [ref]
                cbase_list = cbase_list + cbase

    pwm = pd.DataFrame({'chrom':chrom_list,
                        'pos':pos_list,
                        'ref':ref_list,
                        'baseq':baseq_list,
                        'mapq':mapq_list,
                        'chrom':chrom_list,
                        'A':A_list,
                        'T':T_list,
                        'G':G_list,
                        'C':C_list,
                        'cbase':cbase_list
                       })
    pwm['match'] = [x==y for x,y in zip(pwm['ref'], pwm['cbase'])]
    
    return pwm

In [203]:
min_cov = None
if min_cov == None:
    print('yes')

yes


In [169]:
aligner = 'STAR'

if aligner == 'STAR':
    output_dir = '/mnt/ibm_lg/daniel_le/data/botryllus/results/'
elif aligner == 'BWA':
    output_dir = '/mnt/ibm_lg/daniel_le/data/botryllus/coverage/bwa_coverage/'

completed_targets = glob.glob(f'{output_dir}*.bam')
completed_targets


['/mnt/ibm_lg/daniel_le/data/botryllus/results/sample_49_Aligned.sortedByCoord.out.bam',
 '/mnt/ibm_lg/daniel_le/data/botryllus/results/sample_34_Aligned.sortedByCoord.out.bam',
 '/mnt/ibm_lg/daniel_le/data/botryllus/results/sample_87_Aligned.sortedByCoord.out.bam',
 '/mnt/ibm_lg/daniel_le/data/botryllus/results/sample_48_Aligned.sortedByCoord.out.bam',
 '/mnt/ibm_lg/daniel_le/data/botryllus/results/sample_16_Aligned.sortedByCoord.out.bam',
 '/mnt/ibm_lg/daniel_le/data/botryllus/results/sample_93_Aligned.sortedByCoord.out.bam',
 '/mnt/ibm_lg/daniel_le/data/botryllus/results/sample_58_Aligned.sortedByCoord.out.bam',
 '/mnt/ibm_lg/daniel_le/data/botryllus/results/sample_94_Aligned.sortedByCoord.out.bam',
 '/mnt/ibm_lg/daniel_le/data/botryllus/results/sample_73_Aligned.sortedByCoord.out.bam',
 '/mnt/ibm_lg/daniel_le/data/botryllus/results/sample_72_Aligned.sortedByCoord.out.bam',
 '/mnt/ibm_lg/daniel_le/data/botryllus/results/sample_15_Aligned.sortedByCoord.out.bam',
 '/mnt/ibm_lg/daniel_

In [235]:
sample_index = 75

fasta_path = '/mnt/ibm_lg/daniel_le/data/botryllus/genome/botznik-chr.fa'
genebed_path = '/mnt/ibm_lg/daniel_le/data/botryllus/genome/botznik-chr-all.bed'
if aligner == 'STAR':
    bam_path = f'{output_dir}sample_{sample_index}_Aligned.sortedByCoord.out.bam'
    bw_path = f'{output_dir}sample_{sample_index}_Aligned.sortedByCoord.out.bam.bw'
elif aligner == 'BWA':
    bam_path = f'{output_dir}/bwa_alignments/sample_{sample_index}.sorted.bam'
    bw_path = f'{output_dir}sample_{sample_index}_bs1.bw'
    
dataset_paths = [
                 bam_path,
                 bw_path,
                 genebed_path,
                ]

# BHF = chr9:7538434..7544266
chrom = "chr9"
start = 7538434
end =   7544266

# create pileup
if aligner == 'BWA':
    cov_dir = '/mnt/ibm_lg/daniel_le/data/botryllus/coverage/bwa_slice_cov/'
elif aligner == 'BWA':
    cov_dir = '/mnt/ibm_lg/daniel_le/data/botryllus/coverage/star_slice_cov/'
cov_path = f'{cov_dir}sample_{sample_index}_{chrom}:{start}-{end}.pileup'
    
if not os.path.exists(cov_path):
    print('create pileup...')
    subprocess.call(['samtools',
                     'mpileup',
                     '-s',
                     '-f',
                     fasta_path,
                     '-r',
                     f'{chrom}:{start}-{end}',
                     bam_path,
                     '-o',
                     cov_path
                    ])

# doc = genomeview.visualize_data(dataset_paths, chrom, start, end, fasta_path)
# doc


In [215]:
pwm = create_pwm(cov_path)
pwm[pwm['match'] == False]

# align isoforms to either update genome or annotation file for full length BHF
# Look for inverted repeat regions (see ppt)




coverage_threshold =  126


,chrom,pos,ref,baseq,mapq,A,T,G,C,cbase,match
945,chr9,7539379,C,37,60,29,128,0,25,T,False
950,chr9,7539384,T,37,60,0,21,37,153,C,False
1609,chr9,7540043,C,37,60,152,495,0,0,A/T,False
1613,chr9,7540047,A,37,60,226,0,329,139,A/C/G,False
1640,chr9,7540074,A,37,60,1,0,263,611,C/G,False
1657,chr9,7540091,T,37,60,2,2,648,280,C/G,False
1912,chr9,7540346,N,37,60,17,9,1314,493,C/G,False
2104,chr9,7540538,A,37,60,1139,0,776,462,A/C/G,False
2168,chr9,7540602,G,37,60,466,803,1162,1,A/G/T,False
2401,chr9,7540835,A,37,60,1542,186,0,0,A/T,False


In [216]:
# extract split reads
# create bigwig and pileup

output_dir = '/mnt/ibm_lg/daniel_le/data/botryllus/results/'
completed_targets = glob.glob(f'{output_dir}*.bam')
completed_targets


['/mnt/ibm_lg/daniel_le/data/botryllus/results/sample_49_Aligned.sortedByCoord.out.bam',
 '/mnt/ibm_lg/daniel_le/data/botryllus/results/sample_34_Aligned.sortedByCoord.out.bam',
 '/mnt/ibm_lg/daniel_le/data/botryllus/results/sample_87_Aligned.sortedByCoord.out.bam',
 '/mnt/ibm_lg/daniel_le/data/botryllus/results/sample_48_Aligned.sortedByCoord.out.bam',
 '/mnt/ibm_lg/daniel_le/data/botryllus/results/sample_16_Aligned.sortedByCoord.out.bam',
 '/mnt/ibm_lg/daniel_le/data/botryllus/results/sample_93_Aligned.sortedByCoord.out.bam',
 '/mnt/ibm_lg/daniel_le/data/botryllus/results/sample_58_Aligned.sortedByCoord.out.bam',
 '/mnt/ibm_lg/daniel_le/data/botryllus/results/sample_94_Aligned.sortedByCoord.out.bam',
 '/mnt/ibm_lg/daniel_le/data/botryllus/results/sample_73_Aligned.sortedByCoord.out.bam',
 '/mnt/ibm_lg/daniel_le/data/botryllus/results/sample_72_Aligned.sortedByCoord.out.bam',
 '/mnt/ibm_lg/daniel_le/data/botryllus/results/sample_15_Aligned.sortedByCoord.out.bam',
 '/mnt/ibm_lg/daniel_

In [ ]:
n_threads = 16
tmp_dir = '/mnt/ibm_lg/daniel_le/data/botryllus/tmp_splits/'
output_splits_dir = '/mnt/ibm_lg/daniel_le/data/botryllus/star_splits/'

target = f'/mnt/ibm_lg/daniel_le/data/botryllus/results/sample_{sample_index}_Aligned.sortedByCoord.out.bam'
target_fn_prefix = target.split('/')[-1].split('.bam')[0]
output_splitbam = f'{output_splits_dir}{target_fn_prefix}.splitBam.bam'
output_splitbw = f'{output_splits_dir}{target_fn_prefix}.splitBam.bw'
output_splitpileup = f'{output_splits_dir}{target_fn_prefix}.splitBam.pileup'

# filter bam file for split reads
with open(output_splitbam, 'w') as outfile:
    p1 = subprocess.Popen(['samtools',
                         'view',
                           '-h',
                         '-F', # only unique reads
                         '256',
                         target,
                         f'{chrom}:{start}-{end}',
                        ], shell=False, stdin=subprocess.PIPE, stdout=subprocess.PIPE)
    p2 = subprocess.Popen([f"awk '$6 ~ /N/ && $4 >= {start} || $1 ~ /^@/'",
                          ], shell=True, stdin=p1.stdout, stdout=subprocess.PIPE)
    p3 = subprocess.Popen(['samtools',
                             'view',
                             '-S',
                             '-b',
                             '-',
                            ], shell=False, stdin=p2.stdout, stdout=outfile)

# index bam
bai_path = f'{output_splitbam}.bai'
subprocess.call(['samtools',
                 'index',
                 output_splitbam,
                ])
    
# create bigwig track
bin_size = 1
subprocess.call(['bamCoverage',
                '-p',
                str(n_threads),
                '--normalizeUsing',
                'CPM',
                '--ignoreDuplicates',
                '--binSize',
                str(bin_size),
                '-b',
                output_splitbam,
                '-o',
                output_splitbw,
                ])

# create pileup
subprocess.call(['samtools',
                 'mpileup',
                 '-s',
                 '-f',
                 fasta_path,
                 '-r',
                 f'{chrom}:{start}-{end}',
                 output_splitbam,
                 '-o',
                 output_splitpileup
                ])
    

In [ ]:
dataset_paths = [
                 output_splitbam,
                 output_splitbw,
                 genebed_path,
                ]

doc = genomeview.visualize_data(dataset_paths, chrom, start, end, fasta_path)
display(doc)

pwm = create_pwm(output_splitpileup, min_cov = 1)
pwm[[a!=0 or t!=0 or g!=0 or c!=0 for a,t,g,c in zip(pwm['A'],
                                                        pwm['T'],
                                                        pwm['G'],
                                                        pwm['C'],)]]